In [1]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import statsmodels as sm
import statsmodels.api as smapi
import matplotlib.pyplot as plt

from numpy.linalg import eig

from util.ZigZag0 import ZigZag
from matplotlib.animation import FuncAnimation

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [2]:
btc = np.array(pd.read_csv('bitfinex_BTCUSD5m.csv', header=0).Close)
dsh = np.array(pd.read_csv('bitfinex_DSHUSD5m.csv', header=0).Close)
eth = np.array(pd.read_csv('bitfinex_ETHUSD5m.csv', header=0).Close)

prices = pd.DataFrame(np.column_stack((btc, dsh, eth)))
returns = prices.pct_change().dropna()

In [3]:
Y = np.copy(returns.values)
(row, col) = np.shape(Y)

In [4]:
for j in range(0,col):
    mean = np.mean(Y[:,j])
    std = np.std(Y[:,j])
    
    for i in range(0, row):
        Y[i,j] = (Y[i,j] - mean) / std

In [5]:
corr = np.dot(Y.T, Y) / (row - 1)

In [6]:
eigval, eigv = np.linalg.eig(corr)
sortedindex = eigval.argsort()[::-1]
eigv = eigv[:, sortedindex]

In [7]:
cumeigval = np.cumsum(eigval)
plt.plot(cumeigval)

In [8]:
eigv = eigv[:, :1]
eigvec = np.zeros(np.shape(eigv))

for i in range(0, np.shape(eigv)[0]):
    for j in range(0, np.shape(eigv)[1]):
        eigvec[i,j] = eigv[i,j].real / np.std(returns.values[:,j])
        
for i in range(0,np.shape(eigvec)[1]):
    eigvec[:,i] = eigvec[:,i] / np.sum(np.abs(eigvec[:,j]))

eigret = np.dot(returns.values, eigvec)

print eigvec


[[ 0.34921267]
 [ 0.31101515]
 [ 0.33977218]]


In [9]:
def fit_OU(ts):
    X = smapi.add_constant(ts[:-1])
    model = smapi.OLS(ts[1:], X)
    results = model.fit()
    b = results.params[1]
    a = results.params[0]
    e = np.std(results.resid)
    k = -math.log(b) * 252
    var = e**2 / (2*k) * (1 - math.exp(-2 * k))
    num = -a * np.sqrt(1-b**2)
    den = (1-b) * np.sqrt(var)
    return num / den, 0. 

In [10]:
%matplotlib

S = []
for s in returns:
    # regress stock returns on eigen portfolio returns
    results = smapi.OLS(returns[s].values, smapi.add_constant(eigret)).fit()
    b = results.params
    cumresid = np.cumsum(results.resid)
    score, hf = fit_OU(cumresid) # fit OU    
    S.append(cumresid)
    
    #if abs(score) > 2 and sm.tsa.stattools.adfuller(cumresid)[1] < 0.01:
    #plt.plot(np.cumsum(results.resid))
    #break

Using matplotlib backend: TkAgg


In [22]:
%matplotlib

plt.plot(S[1])



Using matplotlib backend: TkAgg
